In [ ]:
all_text = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_EBA.txt', 'r').readlines()

In [ ]:
q_value_list = []
for line in all_text[1:]:
    q_value_list.append(float(line.split('\t')[-1].strip()))

In [ ]:
from pprint import pprint
print(len(all_text_b))
pprint([a.split('\t') for a in all_text_b])

In [ ]:
all_text_b = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines()


In [ ]:
q_value_list_b = []
for line in all_text_b[1:]:
    q_value_list_b.append(float(line.split('\t')[-1].strip()))

In [ ]:
import pylab as plt
plt.figure(figsize=(8,6))
plt.plot(q_value_list_b, q_value_list, label='random')
plt.plot([0,0.4], [0,0.4], 'k--')
plt.xlabel('True')
plt.ylabel('Estimated')
plt.legend()
plt.grid()

In [ ]:
#calculating the true q_values
con_peaks_all = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-GnpsDecoyConditionalPeaks.txt', 'r').readlines()
cond_peaks_scores = []
for line in con_peaks_all[1:]:
    cond_peaks_scores.append(float(line.split('\t')[-1].strip()))

In [ ]:
print(cond_peaks_scores)

In [ ]:
from pprint import pprint
pprint([a.split('\t') for a in con_peaks_all])

In [ ]:
def calculate_q_value(scores):
    fdr_vals = []
    for i in range(len(scores)):
        fdr = sum(scores[:i])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    return q_vals

In [ ]:
calculate_q_value(cond_peaks_scores)

In [ ]:
def fdr_q(fdr_table):
    fdr_table.sort(key=lambda x: x[1], reverse=True)
    fdr_vals = []
    queries,s,ss = zip(*fdr_table)
    for i in range(len(fdr_table)):
        fdr = sum(ss[:i])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,q in enumerate(queries):
        if ss[i] == 0: # i.e. the true one
            q_list.append((q, q_vals[i]))
    return fdr_vals, q_vals, q_list

In [ ]:
"1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"=="1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"

In [ ]:
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp
pre_spectrums = pp.PassatuttoParser(r'C:\Users\Gosia\Downloads\MassbankOrbi').parse_folder()

In [ ]:
from matchms.importing.load_from_json import as_spectrum
spectrums = []
for i, s in enumerate( pre_spectrums ):
    spectrums.append(as_spectrum(s))
    if i and i % 10 == 0:
        print('processed %d'%i)

In [ ]:
file = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps.txt', 'r').readlines()